##### Join generated page-view uusum features into users in events ######
- 1. Select unique uuid from events 
- 2. Partition and join seperately 00 - 99 
- 3. Append together to be unique events uuid - features
- 4. Join onto events -> join onto train/test by display_id 

In [15]:
###### Import modules 
import pandas as pd
import numpy as np
import time, gc, logging, argparse

In [4]:
###### Read events data 
events = pd.read_csv("../data/events.csv.gz", header = 0)

/home/ubuntu/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#### Check events
events.head()

,display_id,uuid,document_id,timestamp,platform,geo_location
0,1,cb8c55702adb93,379743,61,3,US>SC>519
1,2,79a85fa78311b9,1794259,81,2,US>CA>807
2,3,822932ce3d8757,1179111,182,2,US>MI>505
3,4,85281d0a49f7ac,1777797,234,2,US>WV>564
4,5,8d0daef4bf5b56,252458,338,2,SG>00


In [6]:
#### Hash uuid 
hash_length = 100
def get_hash_code(x):
    y = abs(hash(x)%hash_length)
    return int(y)

events.loc[:,"uuid_hash_code"] = events["uuid"].apply(get_hash_code)
events.head()

,display_id,uuid,document_id,timestamp,platform,geo_location,uuid_hash_code
0,1,cb8c55702adb93,379743,61,3,US>SC>519,80
1,2,79a85fa78311b9,1794259,81,2,US>CA>807,2
2,3,822932ce3d8757,1179111,182,2,US>MI>505,14
3,4,85281d0a49f7ac,1777797,234,2,US>WV>564,59
4,5,8d0daef4bf5b56,252458,338,2,SG>00,90


In [8]:
###### Only keep unique users 
events_uu = events.loc[:,['uuid','uuid_hash_code']].drop_duplicates()
events_uu.sort(['uuid']).head()

/home/ubuntu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


,uuid,uuid_hash_code
3712739,1000003a6ad6e3,36
22230845,10000119e80a30,45
4809328,100004a10b7a20,75
7062300,100004d27b9647,6
6976594,100005b20f8362,53


In [9]:
print events.shape
print events_uu.shape

(23120126, 7)
(19794967, 2)


In [20]:
uu_hash = 0
pageview_uusum = pd.read_csv('../data/pages_view_feature/page_views_uusum_uid' + str(uu_hash) + '.csv.gz', header = 0)
pageview_uusum.head()


,uuid,num_visits,platform-freq,geo_location-freq,traffic_source-freq,source_id-freq,publisher_id-freq,topic_topI-freq,categ_topI-freq,entity_topI-freq
0,100001174c8854,3,1:3,US>CA>803:3,1:3,5315:1|574:1|694:1,1046:1|1161:2,147:1|183:1|265:1,1408:1|1902:1|1909:1,-99:1|3e2329d0532b88665cdd7c9625786738:1|543a1...
1,1000018be5d703,1,2:1,US>FL>528:1,1:1,663:1,61:1,216:1,1702:1,9ebd449f947b6a9358f11aa77f3ce0d2:1
2,10000380c5ae26,3,2:3,US>CA>803:2|US>CA>825:1,1:3,1893:1|465:1|752:1,160:2|43:1,184:1|51:1|77:1,1702:1|1902:1|1908:1,-99:1|9e6e6bfa404b26cfb3012bd457fbdb68:1|f53e4...
3,10000489b154a3,1,1:1,US>MI>563:1,1:1,7344:1,874:1,45:1,1903:1,-99:1
4,100007fce4ea8f,1,1:1,US>MA>506:1,1:1,7343:1,874:1,147:1,2006:1,06c0312fcb3a1445daeb4efd64952503:1


In [21]:
print "check missing", pageview_uusum.isnull().any().sum()
print pageview_uusum.shape

check missing 0
(7169524, 10)


In [16]:
#### Join and append ####
events_uusum_feature = None
uu_hash_used = [0] # 0,1,...,99
for uu_hash in uu_hash_used:
    start_time = time.time() 
    print('join on uu hash %i' %(uu_hash))
    # Read pageview-uusum-feature 
    pageview_uusum = pd.read_csv('../data/pages_view_feature/page_views_uusum_uid' + str(uu_hash) + '.csv.gz', header = 0)
    # Select partition in events_uu
    events_uu_part = events_uu.loc[events_uu.uuid_hash_code == uu_hash, :]
    # Join on uuid
    events_uu_part = events_uu_part.merge(pageview_uusum, how = 'left', on = 'uuid', suffixes = ['','_right'])
    # Check for missing
    print "check missing", events_uu_part.isnull().any().sum()
    # concatenate
    if events_uusum_feature is None:
        events_uusum_feature = events_uu_part
    else:
        events_uusum_feature = pd.concat([events_uusum_feature,events_uu_part], axis=0)
    
    # Clear memory
    del pageview_uusum, events_uu_part
    gc.collect()
    print('It took %i seconds to process uu hash %i' % (time.time()-start_time, uu_hash) )

join on uu hash 0
check missing uuid                   False
uuid_hash_code         False
num_visits             False
platform-freq          False
geo_location-freq      False
traffic_source-freq    False
source_id-freq         False
publisher_id-freq      False
topic_topI-freq        False
categ_topI-freq        False
entity_topI-freq       False
dtype: bool
It took 33 seconds to process uu hash 0


In [18]:
print events_uusum_feature.shape
events_uusum_feature.head()

(197747, 11)


,uuid,uuid_hash_code,num_visits,platform-freq,geo_location-freq,traffic_source-freq,source_id-freq,publisher_id-freq,topic_topI-freq,categ_topI-freq,entity_topI-freq
0,c419799a427c72,0,3,3:3,US>HI>744:3,1:3,1014:1|2734:2,38:1|863:2,128:1|26:1|297:1,2100:3,6837babadc7ee40652ecdb5399e6a207:1|d43ef0ce148...
1,f8198611130910,0,74,1:74,US>MD>512:74,1:68|2:3|3:3,10198:1|1059:1|148:1|1803:1|1886:1|2686:1|2799...,1046:1|1084:1|1175:1|1221:50|158:1|160:2|180:3...,-99:17|10:1|111:1|125:2|140:5|147:1|15:1|160:1...,-99:9|1403:3|1405:1|1406:1|1407:1|1408:1|1607:...,-99:45|034008df3f5e6cff40da618ee25fef76:1|22af...
2,3abf1630ff91c5,0,8,1:8,CA>BC:8,1:4|2:1|3:3,13858:1|2591:1|5792:1|7734:1|7736:1|822:1|9135...,1042:1|1064:1|202:1|253:1|450:2|704:2,130:1|16:1|170:1|181:1|20:1|49:1|65:1|8:1,1408:1|1510:1|1515:1|1609:1|1610:1|1611:1|1709...,-99:2|3d6c006d132611e4ee097cb5814a42b4:1|7d0cd...
3,a829cd392e8574,0,3,2:3,US>CA>807:3,1:3,1809:3,158:3,66:3,1611:3,504e40cfc426912d179d7e436c4f945b:3
4,5ab105d8e0defb,0,1,1:1,AE>03:1,1:1,8803:1,1157:1,26:1,1403:1,-99:1
